<a href="https://colab.research.google.com/github/NiclasFenton-Wiegleb/schlager-lyrics-bot/blob/main/Schlage_bot_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.3
!pip install datasets
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install trl
!pip install safetensors


In [2]:
#Check CUDA is running on GPU

import torch
print(torch.cuda.is_available(), torch.cuda.is_bf16_supported())

True False


In [ ]:
#Import dependencies

from random import randrange
import pandas as pd
from datasets import Dataset

In [ ]:
df_lyrics = pd.read_csv('schlager_songs_v2.csv')

df_lyrics['lyrics'][10]

'{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}'

In [ ]:
# Create separate column for 1st verse

verse_list = []

for ind in df_lyrics["lyrics"].index:
    lyrics = df_lyrics["lyrics"].iloc[ind]
    if lyrics == '{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}' :
        verse_list.append(None)
    else:
        try:
            verse = lyrics.split("\n")[0]
            n = 1
            while len(' '.join(verse).split()) <= 15:
                verse = lyrics.split("\n")[0:n]
                n += 1
            verse_list.append(''.join(verse))

        except:
            verse_list.append(None)


df_lyrics["verse_1"] = verse_list

df_lyrics["verse_1"][0]

df_lyrics.to_csv("schlager_songs_v2.csv")

In [ ]:
# Prompt Example

### Instruction:

"Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben."

### Input:

"Du sagst nicht ein Wort"

### Response:

f""" Du sagst nicht ein Wort
 Und deine Hand wischt eine Träne fort
 Und dein leerer Blick
 Sinkt in dein Glas
 Du sitzt hier vor mir
 Und dein Gesicht lässt keinen Zweifel mehr
 Heut sagst du mir
 Dass ich dich verlier'
 Nie war Zeit für dich
 Ich lebte nur in meiner eignen Welt
 Ich weiß, du wirst gehen
 Ich muss dich versteh'n
 Lieb mich ein letztes mal
 Es bleibt mir keine andre Wahl
 Ich weiß, dass ich die Nacht mit dir
 An den Tag verlier'
...
 Und bleib bei mir"""

" Du sagst nicht ein Wort\n Und deine Hand wischt eine Träne fort\n Und dein leerer Blick\n Sinkt in dein Glas\n Du sitzt hier vor mir\n Und dein Gesicht lässt keinen Zweifel mehr\n Heut sagst du mir\n Dass ich dich verlier'\n Nie war Zeit für dich\n Ich lebte nur in meiner eignen Welt\n Ich weiß, du wirst gehen\n Ich muss dich versteh'n\n Lieb mich ein letztes mal\n Es bleibt mir keine andre Wahl\n Ich weiß, dass ich die Nacht mit dir\n An den Tag verlier'\n...\n Und bleib bei mir"

In [ ]:
def format_instruction(sample):
    return f"""### Instruction:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

{sample['input']}

### Response:

{sample['output']}

"""

In [ ]:
#Create input and output dataset from dataframe

df_dataset = df_lyrics[["verse_1", "lyrics"]]

ind_nan = []

for ind in df_dataset.index:
    if df_dataset["verse_1"].iloc[ind] == None:
        ind_nan.append(ind)
    else:
        continue

df_dataset.drop(index=ind_nan, inplace=True)

df_dataset.reset_index(drop=True, inplace=True)

df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)

dataset = Dataset.from_pandas(df_dataset)

dataset

<ipython-input-8-50f732c958cc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset.drop(index=ind_nan, inplace=True)
<ipython-input-8-50f732c958cc>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)


Dataset({
    features: ['input', 'output'],
    num_rows: 1046
})

In [ ]:
print(format_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

 Ein Tag wie jeder, ich träum' von Liebe

### Response:

 Ein Tag wie jeder, ich träum' von Liebe
 Doch eben nur ein Traum, aha, aha
 Menschen wohin ich schau', Großstadtgetriebe
 Und auf einmal sah ich sie, sie
  Jahr', blondes Haar, so stand sie vor mir (woahoh)
  Jahr', blondes Haar, wie find' ich zu ihr?
 Lalala, lalala, lalalalala
 Sie hat mich angelacht und war vorüber
 Da war's um mich geschehen, aha, aha
 Menschen wohin ich schau', Großstadtgetriebe
 Und überall such ich sie, sie
  Jahr', blondes Haar, so stand sie vor mir (woahoh)
  Jahr', blondes Haar, wie find' ich zu ihr?
 Lalala, lalala, lalalalala
 Lalala, lalala, lalalalala
  Jahr', blondes Haar, so stand sie vor mir (woahoh)
  Jahr', blondes Haar, wie find' ich zu ihr?
  Jahr', blondes Haar, so stand sie vor mir





In [ ]:
# Import dependencies

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import accelerate

#SFTTrainer supports a native integration with peft, which makes it easy to instruction tune LLMs

from peft import LoraConfig, get_peft_model #, prepare_model_for_kbit_training

ImportError: ignored

In [ ]:
# Hugging Face model id
model_id = "malteos/bloom-6b4-clp-german" # non-gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


model = AutoModelForCausalLM.from_pretrained(
      model_id,
      quantization_config=bnb_config,
      use_cache=False,
      device_map="auto")
model.config.pretraining_tp = 1


In [ ]:
#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
#We need to define the hyperparameters before we can start training

args = TrainingArguments(
    output_dir="./content/drive/MyDrive/AiCore/trained_model/New folder",
    num_train_epochs=3,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True  # disable tqdm since with packing values are in correct
)

ValueError: ignored

In [ ]:
#We can now configure the trainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)

In [ ]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()